In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv('data/WeatherPy_challenge.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rainfall in. (last 3 hrs.),Snowfall in. (last 3 hrs.)
0,0,Mahon,IE,2019-11-08 01:24:16,51.89,-8.40,41.00,93,40,12.75,scattered clouds,0.00,0.0
1,1,Jamestown,AU,2019-11-08 01:24:16,-33.21,138.60,51.26,63,87,23.15,light rain,0.19,0.0
2,2,Carire,BR,2019-11-08 01:24:17,-3.95,-40.47,79.88,62,76,6.49,broken clouds,0.00,0.0
3,3,Rio Grande,BR,2019-11-08 01:21:39,-32.03,-52.10,66.38,88,0,7.05,clear sky,0.00,0.0
4,4,Ushuaia,AR,2019-11-08 01:20:42,-54.81,-68.31,37.40,74,75,20.80,shower rain,0.00,0.0


In [4]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

# Ask for snow and rain preferences
rain = input('Do you want it to be raining? (yes/no) ')
snow = input('Do you want it to be snowing? (yes/no) ')

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [5]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]

if rain == 'yes':
    drop_indexes = preferred_cities_df.loc[~preferred_cities_df['Weather Description'].str.contains('rain')].index
else:
    drop_indexes = preferred_cities_df.loc[preferred_cities_df['Weather Description'].str.contains('rain')].index

# Delete these row indexes from dataFrame
preferred_cities_df.drop(drop_indexes, inplace=True)

if snow == 'yes':
    drop_indexes = preferred_cities_df.loc[~preferred_cities_df['Weather Description'].str.contains('snow')].index
else:
    drop_indexes = preferred_cities_df.loc[preferred_cities_df['Weather Description'].str.contains('snow')].index

# Delete these row indexes from dataFrame
preferred_cities_df.drop(drop_indexes, inplace=True)

preferred_cities_df.head(10)

/Users/petergish/opt/anaconda3/envs/pythondata/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rainfall in. (last 3 hrs.),Snowfall in. (last 3 hrs.)
2,2,Carire,BR,2019-11-08 01:24:17,-3.95,-40.47,79.88,62,76,6.49,broken clouds,0.0,0.0
6,6,Victoria,BN,2019-11-08 01:24:07,5.28,115.24,82.40,83,75,4.70,broken clouds,0.0,0.0
10,10,Hilo,US,2019-11-08 01:24:00,19.71,-155.08,84.20,62,1,8.05,clear sky,0.0,0.0
22,22,Lorengau,PG,2019-11-08 01:24:24,-2.02,147.27,80.74,80,100,12.10,overcast clouds,0.0,0.0
29,29,Nelson Bay,AU,2019-11-08 01:24:25,-32.72,152.14,86.00,19,0,27.51,clear sky,0.0,0.0
31,31,Airai,TL,2019-11-08 01:24:26,-8.93,125.41,80.40,39,0,6.55,clear sky,0.0,0.0
33,33,Pundaguitan,PH,2019-11-08 01:24:26,6.37,126.17,82.69,72,99,6.62,overcast clouds,0.0,0.0
35,35,Benguela,AO,2019-11-08 01:24:26,-12.58,13.40,77.18,86,100,2.89,overcast clouds,0.0,0.0
37,37,Kapaa,US,2019-11-08 01:24:27,22.08,-159.32,82.40,61,90,11.41,overcast clouds,0.0,0.0
39,39,Bluff,AU,2019-11-08 01:24:27,-23.58,149.07,88.54,25,0,10.78,clear sky,0.0,0.0


In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Weather Description', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Carire,BR,79.88,broken clouds,-3.95,-40.47,
6,Victoria,BN,82.40,broken clouds,5.28,115.24,
10,Hilo,US,84.20,clear sky,19.71,-155.08,
22,Lorengau,PG,80.74,overcast clouds,-2.02,147.27,
29,Nelson Bay,AU,86.00,clear sky,-32.72,152.14,
31,Airai,TL,80.40,clear sky,-8.93,125.41,
33,Pundaguitan,PH,82.69,overcast clouds,6.37,126.17,
35,Benguela,AO,77.18,overcast clouds,-12.58,13.40,
37,Kapaa,US,82.40,overcast clouds,22.08,-159.32,
39,Bluff,AU,88.54,clear sky,-23.58,149.07,


In [7]:
# Set parameters to search for a hotel.
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

In [8]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row['Lat']
    lng = row['Lng']

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params['location'] = f'{lat},{lng}'

    # Use the search term: 'lodging' and our latitude and longitude.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Carire,BR,79.88,broken clouds,-3.95,-40.47,
6,Victoria,BN,82.40,broken clouds,5.28,115.24,Tiara Labuan Hotel
10,Hilo,US,84.20,clear sky,19.71,-155.08,Hilo Seaside Hotel
22,Lorengau,PG,80.74,overcast clouds,-2.02,147.27,Lorengau Harbourside Hotel
29,Nelson Bay,AU,86.00,clear sky,-32.72,152.14,Mantra Nelson Bay


In [10]:
# Template for pop-up marker
info_box_template = '''
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
'''

In [11]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [12]:
# Add map with a pop-up marker for each city.
locations = hotel_df[['Lat', 'Lng']]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Create the output file (CSV).
output_data_file = 'data/WeatherPy_vacation.csv'
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label='City_ID')